In [1]:
# Necessary Import Statements
import pandas as pd
import numpy as np

# Datasets converted to Dataframes
#BetaData = pd.read_csv('~/ASDRP/Data/BetaData.csv', low_memory=False)
#data = pd.read_csv('~/ASDRP/Data/GammaData.csv', low_memory=False)
data = pd.read_csv('PS_2021.01.18_09.15.37.csv', low_memory=False)

# Initializing dataframes
duplicates = pd.DataFrame(columns=list(data.columns))
deDupedData = pd.DataFrame(columns=list(data.columns))

# Removing entries with no dates
data.dropna(how='any', subset=['pl_pubdate'], inplace=True)

# Removing entries with bad dates
for index, row in data.iterrows():
    date = data['pl_pubdate'][index]
    try:
        if len(date) < 8:
            data['pl_pubdate'][index] = date + '-01'
        data['pl_pubdate'][index] = pd.to_datetime(data['pl_pubdate'][index]) # This line is redundant, remove either this or line 56
    except:
        data.drop([index], inplace=True)
        data.reset_index(drop=True)

# Converting dates to datetime objects
data['pl_pubdate'] = pd.to_datetime(data['pl_pubdate'])

# Sorting the data
sortedData = data.sort_values(by=['pl_name', 'pl_pubdate'], ascending= [1,0], inplace=False, na_position='last')

# Store the duplicates before removing them
duplicates = pd.DataFrame(columns=list(data.columns))
mask = sortedData.duplicated(subset='pl_name', keep='first')
df_keep = sortedData.loc[~mask]
duplicates = duplicates.append(sortedData.loc[mask])

# Dropping the duplicates
deDupedData = sortedData.drop_duplicates(subset = 'pl_name', keep='first', inplace=False, ignore_index=True)

# Printing the first 10 entries of the DeDupedData
print(deDupedData[['pl_name', 'pl_pubdate']][0:10])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


                   pl_name pl_pubdate
0                 11 Com b 2011-08-01
1                 11 UMi b 2017-03-01
2                 14 And b 2011-08-01
3                 14 Her b 2017-03-01
4               16 Cyg B b 2017-03-01
5                 18 Del b 2011-08-01
6  1RXS J160929.1-210524 b 2015-03-01
7                 24 Boo b 2018-08-01
8                 24 Sex b 2011-01-01
9                 24 Sex c 2011-01-01


In [2]:
# Imports the datasets
phl = pd.read_csv('PHL_Dataset.csv', low_memory=False)

In [3]:
# Removes any planets that have an NA eccentricity value
deDupedData = deDupedData[deDupedData['pl_orbeccen'].notna()]
deDupedData = deDupedData[deDupedData['pl_rade'].notna()]

# Resets the data row index
deDupedData = deDupedData.reset_index (drop = True)

In [4]:
# Makes an empty m_type dataframe with same column headers
m_type = pd.DataFrame(columns=list(data.columns))

# Makes an empty m_type_habitable dataframe to store planets that result out of this code
m_type_habitable = pd.DataFrame(columns=list(data.columns))

# Fills the m_type dataframe with M-type host stars' planets
m_type = deDupedData.loc[(deDupedData['st_spectype'].str.startswith('M', na=False))]

# Resets the m_type row index
m_type = m_type.reset_index (drop = True)

In [5]:
# Takes out planets that have no planet radius and eccentricity
phl = phl[phl['pl_rade'].notna()]
phl = phl[phl['pl_orbeccen'].notna()]

# Resets index
phl = phl.reset_index (drop = True)

In [6]:
# Makes a new empty dataframe for the m-type planets in the PHL dataset
phl_m_type = pd.DataFrame(columns=list(phl.columns))

In [7]:
# Fills the empty dataframe with m-type planets from the PHL dataset
phl_m_type = phl.loc[(phl['st_spectype'].str.startswith('M', na=False))]

# Resets index
phl_m_type = phl_m_type.reset_index (drop = True)

In [8]:
def check_match(planet, control_planet):
    if pd.isna(planet['pl_dens']):
        if (not pd.isna(planet['pl_radj'])) & (not pd.isna(planet['pl_massj'])):
            planet['pl_dens'] = (planet['pl_massj'] * (1.898 * (10 ** 27) * 1000)) \
            / (4 * np.pi * ((planet['pl_radj'] * 43441 * 160934) ** 3) / 3)
        else:
            return
        
    if pd.isna(planet['pl_orbsmax']) and (pd.isna(planet['pl_ratdor']) or pd.isna(planet['st_rad'])):
        return

    # Calculates orbital period in days for main planet
    if pd.isna(planet['pl_orbper']):
        if pd.isna(planet['pl_ratdor']):
            planet['pl_orbper'] = np.sqrt(planet['pl_orbsmax']**3) * 365
        elif not pd.isna(planet['st_rad']): 
            planet['pl_orbper'] = np.sqrt((planet['pl_ratdor']*planet['st_rad'])**3) * 365
        if pd.isna(planet['pl_orbper']):
            return
    
    if pd.isna(control_planet['pl_orbsmax']) and (pd.isna(control_planet['pl_ratdor']) or pd.isna(control_planet['st_rad'])):
        return

    # Calculates orbital period in days for phl planet
    if pd.isna(control_planet['pl_orbper']):
        if pd.isna(control_planet['pl_ratdor']):
            control_planet['pl_orbper'] = np.sqrt(control_planet['pl_orbsmax']**3) * 365
        elif not pd.isna(control_planet['st_rad']): 
            control_planet['pl_orbper'] = np.sqrt((control_planet['pl_ratdor']*control_planet['st_rad'])**3) * 365
        if pd.isna(control_planet['pl_orbper']):
            return
    
    if pd.isna(planet['pl_orbeccen']):
        return
    
    #global x
    #x = x.append(planet, ignore_index = True)

    #global a
    #a = a.append(control_planet, ignore_index = True)
    

    if (float(planet['pl_dens']) >= 4) & (float(planet['pl_dens']) <= 7):
        if (planet['pl_controv_flag'] == '0'):
            if ('Transit' in planet['discoverymethod']):
                if (planet['pl_orbeccen'] <= 0.2):
                    if (np.abs(float(planet['pl_orbper']) - float(control_planet['pl_orbper'])) <= 50):
                        global m_type_habitable
                        m_type_habitable = m_type_habitable.append(planet, ignore_index = True)


In [9]:
closest_distance_away = 10000
index = -1

for i in range (len(m_type['pl_rade'])): 
    current_dist = np.abs(phl_m_type['pl_rade'][0]-m_type['pl_rade'][i])
    if current_dist < 10:
        closest_distance_away = current_dist
        index = 0
        
    current_dist = np.abs(phl_m_type['pl_rade'][1]-m_type['pl_rade'][i])
    if current_dist < 10:
        if closest_distance_away > current_dist:
            closest_distance_away = current_dist
            index = 1
            
    current_dist = np.abs(phl_m_type['pl_rade'][2]-m_type['pl_rade'][i])
    if current_dist < 10:
        if closest_distance_away > current_dist:
            closest_distance_away = current_dist
            index = 2
            
    if index == -1 or closest_distance_away == 10000:
        continue
        
    planet = m_type.loc[i].copy()
    control_planet = phl_m_type.loc[index].copy()
    
    check_match(planet, control_planet)
    closest_distance_away = 10000
    index = -1



In [10]:
m_type_habitable

,rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,default_flag,sy_snum,sy_pnum,sy_mnum,cb_flag,discoverymethod,disc_year,disc_refname,disc_pubdate,disc_locale,disc_facility,disc_telescope,disc_instrument,rv_flag,pul_flag,ptv_flag,tran_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,dkin_flag,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,...,sy_zmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_w3mag,sy_w3magerr1,sy_w3magerr2,sy_w4mag,sy_w4magerr1,sy_w4magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_icmag,sy_icmagerr1,sy_icmagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,rowupdate,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,Unnamed: 287,Unnamed: 288,Unnamed: 289,Unnamed: 290,Unnamed: 291,Unnamed: 292
0,27886,L 98-59 c,L 98-59,c,NaN,NaN,TIC 307210830,Gaia DR2 5271055243163629056,1,1,3,0,0,Transit,2019,<a refstr=KOSTOV_ET_AL__2019 href=https://ui.a...,2019-07,Space,Transiting Exoplanet Survey Satellite (TESS),0.1 m TESS Telescope,TESS CCD Array,1,0,0,1,0,0,0,0,0,0,Published Confirmed,0,<a refstr=CLOUTIER_ET_AL__2019 href=https://ui...,3.6904,0.0003,-0.0003,0,0.03170,0.00010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.5976,0.000765,-0.000765,NaN,NaN,NaN,9.41242,0.007282,-0.007282,NaN,NaN,NaN,2020-04-10,2019-09-01,2020-04-16,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,27907,LTT 3780 b,LTT 3780,b,NaN,NaN,TIC 36724087,Gaia DR2 3767281845873242112,1,2,2,0,0,Transit,2020,<a refstr=CLOUTIER_ET_AL__2020 href=https://ui...,2020-07,Space,Transiting Exoplanet Survey Satellite (TESS),0.1 m TESS Telescope,TESS CCD Array,1,0,0,1,0,0,0,0,0,0,Published Confirmed,0,<a refstr=CLOUTIER_ET_AL__2020 href=https://ui...,0.768448,5.5E-05,-5.3E-05,0,0.01211,0.00012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.8465,0.000477,-0.000477,NaN,NaN,NaN,10.58480,0.007385,-0.007385,NaN,NaN,NaN,2020-05-28,2020-07-01,2020-06-04,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,28370,TOI-1266 b,TOI-1266,b,NaN,NaN,TIC 467179528,Gaia DR2 1678074272650459008,1,1,2,0,0,Transit,2020,<a refstr=DEMORY_ET_AL__2020 href=https://ui.a...,2020-09,Space,Transiting Exoplanet Survey Satellite (TESS),0.1 m TESS Telescope,TESS CCD Array,0,0,0,1,0,0,0,0,0,0,Published Confirmed,0,<a refstr=DEMORY_ET_AL__2020 href=https://ui.a...,10.894843,6.7E-05,-6.6E-05,0,0.07360,0.00160,...,-0.004641,8.715,0.022,-0.022,8.612001,0.019,-0.019,8.504,0.024,-0.024,8.233,0.207,-0.207,12.1222,0.000249,-0.000249,NaN,NaN,NaN,11.04020,0.007315,-0.007315,NaN,NaN,NaN,2020-10-01,2020-09-01,2020-10-08,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
phl_m_type['pl_orbper']

0       37.426
1    32.939614
2     24.73712
Name: pl_orbper, dtype: object

In [ ]:
phl_m_type

,rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,default_flag,sy_snum,sy_pnum,sy_mnum,cb_flag,discoverymethod,disc_year,disc_refname,disc_pubdate,disc_locale,disc_facility,disc_telescope,disc_instrument,rv_flag,pul_flag,ptv_flag,tran_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,dkin_flag,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,...,sy_imagerr2,sy_zmag,sy_zmagerr1,sy_zmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_w3mag,sy_w3magerr1,sy_w3magerr2,sy_w4mag,sy_w4magerr1,sy_w4magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_icmag,sy_icmagerr1,sy_icmagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,rowupdate,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,Unnamed: 287,Unnamed: 288,Unnamed: 289
0,28420,TOI-700 d,TOI-700,d,NaN,NaN,TIC 150428135,Gaia DR2 5284517766615492736,0,1,3,0,0,Transit,2020,<a refstr=GILBERT_ET_AL__2020 href=https://ui....,2020-01,Space,Transiting Exoplanet Survey Satellite (TESS),0.1 m TESS Telescope,TESS CCD Array,0,0,0,1,0,0,0,0,0,0,Published Confirmed,0,<a refstr=GILBERT_ET_AL__2020 href=https://ui....,37.426,0.0007,-0.001000,0.0,0.1630,0.0150,...,NaN,NaN,NaN,NaN,8.523,0.023,-0.023,8.392,0.02,-0.02,8.281,0.019,-0.019,8.234,0.115,-0.115,12.0665,0.000478,-0.000478,NaN,NaN,NaN,10.9102,0.007423,-0.007423,NaN,NaN,NaN,2020-07-17,2020-01,2020-07-23,1,0,0,0,0,0,NaN,NaN,NaN
1,2840,K2-18 b,K2-18,b,NaN,NaN,TIC 388804061,Gaia DR2 3910747531814692736,0,1,2,0,0,Transit,2015,<a refstr=MONTET_ET_AL__2015 href=https://ui.a...,2015-08,Space,K2,0.95 m Kepler Telescope,Kepler CCD Array,1,0,0,1,0,0,0,0,0,0,Published Confirmed,0,<a refstr=BENNEKE_ET_AL__2017 href=https://ui....,32.939614,0.000101,-0.000084,0.0,0.1429,0.0060,...,-0.000807,11.5166,0.004301,-0.004301,8.769,0.023,-0.023,8.668,0.02,-0.02,8.552,0.025,-0.025,8.587,0.379,-0.379,12.4154,0.000502,-0.000502,NaN,NaN,NaN,11.2373,0.007370,-0.007370,12.473,NaN,NaN,2019-03-18,2017-01,2016-11-03,2,0,0,0,0,37,NaN,NaN,NaN
2,27890,LHS 1140 b,LHS 1140,b,NaN,NaN,TIC 92226327,Gaia DR2 2371032916186181760,0,1,2,0,0,Transit,2017,<a refstr=DITTMANN_ET_AL__2017 href=https://ui...,2017-04,Ground,MEarth Project,0.4 m MEarth Telescope,Apogee CCD Sensor,1,0,0,1,0,0,0,0,0,0,Published Confirmed,0,<a refstr=DITTMANN_ET_AL__2017 href=https://ui...,24.73712,0.00025,-0.000250,0.0,0.0875,0.0041,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.6706,0.000356,-0.000356,NaN,NaN,NaN,11.2991,0.007329,-0.007329,NaN,NaN,NaN,2018-12-03,2017-04,2017-04-27,0,0,0,0,0,0,NaN,NaN,NaN


In [ ]:
m_type_habitable['pl_rade']

0    1.160
1    1.350
2    1.332
3    2.370
Name: pl_rade, dtype: float64